In [14]:
import os
import redis
import hashlib
from metaflow import FlowSpec, step, S3
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

# Ejemplo de predicción de modelo usando Redis

Este notebook muestra cómo obtener predicciones de un modelo que produce predicciones en lotes. Las predicciones se cargaron en Redis. 

En este notebook, recuperamos las predicciones del modelo almacenadas en Redis. Los datos deben ser leídos, convertidos a cadenas y luego hasheados. Con este enfoque, podemos verificar si los datos existen en Redis y recuperar la predicción correspondiente. En caso de que los datos no existan, asignamos un valor de cero. Esta estrategia simula cómo podría comportarse un servicio en producción ante casos no contemplados.

La ventaja de utilizar Redis en este contexto radica en su capacidad para almacenar datos de forma eficiente en memoria, lo que permite un acceso rápido a las predicciones previamente calculadas. Además, Redis ofrece funcionalidades de almacenamiento en caché y persistencia de datos, lo que garantiza la disponibilidad y la integridad de las predicciones incluso en entornos de producción de alta demanda.

In [15]:
# Conectamos al servidor redis (asegúrate de que el docker compose esté corriendo)
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Configuración de las credenciales de acceso a AWS S3 (minio)
os.environ['AWS_ACCESS_KEY_ID'] = "minio"
os.environ['AWS_SECRET_ACCESS_KEY'] = "minio123"
os.environ['AWS_ENDPOINT_URL_S3'] = "http://localhost:9000"

In [16]:
# Obtención de datos para prueba

df = pd.read_csv("./breast_cancer.csv", header=None)

# Sampleamos 100 valores al azar
df_temp = df.sample(50)

test_values = df_temp.values.tolist()

s3 = S3(s3root="s3://amqtp/")
scaler_obj = s3.get("scaler.pkl")
with open(scaler_obj.path, 'rb') as f:
    scaler = pickle.load(f)

# Aplicamos el scaler a los datos
scaled_values = scaler.transform(df_temp)

print(scaled_values)

# Conversión de valores a cadenas y hash
# Esto debería implementarse en el pipeline. Dado que los números de punto flotante pueden 
# presentar problemas debido a pequeñas variaciones, se podría considerar redondearlos.
string_values = [' '.join(map(str, sublist)) for sublist in scaled_values]
hashed_values = [hashlib.sha256(substring.encode()).hexdigest() for substring in string_values]

# Inicializamos un diccionario para almacenar las salidas del modelo
model_outputs = {}

# Obtenemos las predicciones almacenadas en Redis
for hash_key in hashed_values:
    model_outputs[hash_key] = r.hgetall(f"predictions:{hash_key}")

# Reemplazo de valores nulos con un valor predeterminado
# Esto es necesario porque, en caso de una nueva entrada que el modelo no haya visto durante 
# el procesamiento por lotes, necesitamos proporcionar una salida. Esta entrada podría ser
# encolada para su posterior procesamiento en lotes.
# model_output = [value if value is not None else "jjjj" for value in model_output]

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: could not convert string to float: 'mean radius'

Veamos la salida del modelo para diferentes entradas:

In [ ]:
print("Salidas de los modelos para las primeras 5 entradas:")
for index, test_value in enumerate(test_values):
    hash_key = hashed_values[index]
    tree_prediction = model_outputs[hash_key].get('tree', 'No disponible')
    svc_prediction = model_outputs[hash_key].get('svc', 'No disponible')
    knn_prediction = model_outputs[hash_key].get('knn', 'No disponible')
    reglog_prediction = model_outputs[hash_key].get('reglog', 'No disponible')
    
    print(f"\nPara la entrada: {test_value}")
    print(f"El modelo tree predice: {tree_prediction}")
    print(f"El modelo svc predice: {svc_prediction}")
    print(f"El modelo knn predice: {knn_prediction}")
    print(f"El modelo reglog predice: {reglog_prediction}")

print("\nSe han mostrado las predicciones para las primeras 5 entradas.")

Salidas de los modelos para las primeras 5 entradas:

Para la entrada: ['9.423', '27.88', '59.26', '271.3', '0.08123', '0.04971', '0.0', '0.0', '0.1742', '0.06059', '0.5375', '2.927', '3.618', '29.11', '0.01159', '0.01124', '0.0', '0.0', '0.03004', '0.003324', '10.49', '34.24', '66.5', '330.6', '0.1073', '0.07158', '0.0', '0.0', '0.2475', '0.06969']
El modelo tree predice: No disponible
El modelo svc predice: No disponible
El modelo knn predice: No disponible
El modelo reglog predice: No disponible

Para la entrada: ['13.46', '18.75', '87.44', '551.1', '0.1075', '0.1138', '0.04201', '0.03152', '0.1723', '0.06317', '0.1998', '0.6068', '1.443', '16.07', '0.004413', '0.01443', '0.01509', '0.007369', '0.01354', '0.001787', '15.35', '25.16', '101.9', '719.8', '0.1624', '0.3124', '0.2654', '0.1427', '0.3518', '0.08665']
El modelo tree predice: No disponible
El modelo svc predice: No disponible
El modelo knn predice: No disponible
El modelo reglog predice: No disponible

Para la entrada: ['16

# Representemos graficamente a estos datos
test_values_df = pd.DataFrame(test_values)
test_values_df = pd.concat([test_values_df, pd.Series(model_output)], axis=1)
test_values_df.columns=["0", "1", "2", "3", "label"]
sns.scatterplot(x="2", y="3", hue="label", data=test_values_df)
plt.title("Model output for test values");

In [ ]:

# Creamos un DataFrame con los valores de test
test_values_df = pd.DataFrame(test_values)

# Obtenemos la lista de modelos disponibles
models = list(set().union(*[set(output.keys()) for output in model_outputs.values()]))

# Agregamos las predicciones de cada modelo al DataFrame
for model in models:
    test_values_df[model] = [model_outputs[hash_key].get(model, 'No disponible') for hash_key in hashed_values]

# Renombramos las columnas
test_values_df.columns = [str(i) for i in range(len(test_values_df.columns) - len(models))] + models

# Convertimos las predicciones a valores numéricos
for model in models:
    test_values_df[model] = pd.to_numeric(test_values_df[model], errors='coerce')

# Creamos los scatter plots
n_cols = 3  # Número de columnas en la cuadrícula de gráficos
n_rows = (len(models) + n_cols - 1) // n_cols  # Número de filas necesarias

plt.figure(figsize=(6*n_cols, 5*n_rows))

for i, model in enumerate(models, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.scatterplot(x="0", y="1", hue=model, data=test_values_df, palette="viridis")
    plt.title(f"Predicciones del modelo {model}")

plt.tight_layout()
plt.show()

<Figure size 1800x0 with 0 Axes>